In [1]:
import json
import pandas as pd
import requests
import time

In [2]:
url = "https://api.opendota.com/api/publicMatches?api_key=062963e6-f4c4-4ae4-9a56-a0136f3c2a18"
response = requests.get(url)

In [3]:
selected_games = []
all_games = []

In [4]:
# for i in range(0,100):
#     matches = requests.get(url).json()
#     print(len(matches))
#     for j in matches:
#         all_games.append(j)
#     time.sleep(3)

In [5]:
# with open('data2.json', 'w') as f:
#     json.dump(all_games, f)

In [6]:
f = open('data.json',)
first=json.load(f)
j = open('data2.json',)
second=json.load(j)
all_games = first + second

In [7]:
len(all_games)

20000

In [8]:
# all = []
# for i in all_games:
#     all = all + list(i["radiant_team"].split(","))
#     all = all + list(i["dire_team"].split(","))

In [9]:
# from collections import Counter
# Counter(all)

In [10]:
# winrate_dict = {}
# boilerdict = {"wins":0,"games":0}

In [11]:
# def addwin(hero_id):
#     if hero_id not in winrate_dict:
#         winrate_dict[hero_id] = boilerdict
#     else:
#         winrate_dict[hero_id]["wins"] +=1
#         winrate_dict[hero_id]["games"] +=1

# def addloss(hero_id):
#     if hero_id not in winrate_dict:
#         winrate_dict[hero_id] = boilerdict
#     else:
#         winrate_dict[hero_id]["games"] +=1

In [12]:
filtered_match = []
winrate_dict = {}
result_list = []
for match in all_games:
    if match["avg_mmr"] and match["game_mode"] == 22 and match["avg_mmr"] > 3000 and match["lobby_type"] == 7:
        # #Pick apart the match and gather the info
        # #determine winning team
        # if match["radiant_win"]:
        #     winners = list(match["radiant_team"].split(","))
        #     losers = list(match["dire_team"].split(","))
        # else:
        #     winners= list(match["dire_team"].split(","))
        #     losers = list(match["radiant_team"].split(","))
        # for hero in winners:
        #     addwin(hero)
        # for hero in losers:
        #     addloss(hero)
        result = int(match["radiant_win"])
        for rhero in list(match["radiant_team"].split(",")):
            for dhero in list(match["dire_team"].split(",")):
                result_list.append([result,rhero,dhero])

In [13]:
result_list

[[1, '45', '58'],
 [1, '45', '50'],
 [1, '45', '14'],
 [1, '45', '44'],
 [1, '45', '16'],
 [1, '86', '58'],
 [1, '86', '50'],
 [1, '86', '14'],
 [1, '86', '44'],
 [1, '86', '16'],
 [1, '54', '58'],
 [1, '54', '50'],
 [1, '54', '14'],
 [1, '54', '44'],
 [1, '54', '16'],
 [1, '136', '58'],
 [1, '136', '50'],
 [1, '136', '14'],
 [1, '136', '44'],
 [1, '136', '16'],
 [1, '26', '58'],
 [1, '26', '50'],
 [1, '26', '14'],
 [1, '26', '44'],
 [1, '26', '16'],
 [0, '49', '74'],
 [0, '49', '14'],
 [0, '49', '87'],
 [0, '49', '84'],
 [0, '49', '70'],
 [0, '5', '74'],
 [0, '5', '14'],
 [0, '5', '87'],
 [0, '5', '84'],
 [0, '5', '70'],
 [0, '54', '74'],
 [0, '54', '14'],
 [0, '54', '87'],
 [0, '54', '84'],
 [0, '54', '70'],
 [0, '76', '74'],
 [0, '76', '14'],
 [0, '76', '87'],
 [0, '76', '84'],
 [0, '76', '70'],
 [0, '64', '74'],
 [0, '64', '14'],
 [0, '64', '87'],
 [0, '64', '84'],
 [0, '64', '70'],
 [1, '29', '98'],
 [1, '29', '106'],
 [1, '29', '30'],
 [1, '29', '86'],
 [1, '29', '70'],
 [1, '128

In [14]:
df = pd.DataFrame(result_list,columns=["winner","hero1","hero2"])
df['combo'] = df['hero1'] + "-" + df['hero2']
df.head()

,winner,hero1,hero2,combo
0,1,45,58,45-58
1,1,45,50,45-50
2,1,45,14,45-14
3,1,45,44,45-44
4,1,45,16,45-16


In [15]:
len(df['hero2'].unique())

118

In [16]:
X = df['combo']
y = df["winner"]
print(X.shape,y.shape)

(144175,) (144175,)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from tensorflow.keras.utils import to_categorical

# X_scaler = MinMaxScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)





In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
X_train_np = X_train.to_numpy().reshape(-1,1)
X_test_np = X_test.to_numpy().reshape(-1,1)
X_np = X.to_numpy().reshape(-1,1)


In [20]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [21]:
ohe = OneHotEncoder()
ohe.fit(X_np)
encoded_X_train = ohe.transform(X_train_np)
encoded_X_test = ohe.transform(X_test_np)


In [22]:
# X_train_cat = to_categorical(encoded_X_train)
# X_test_cat = to_categorical(encoded_X_test)

In [23]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 100)
model.fit(encoded_X_train,y_train_categorical)

RandomForestRegressor()

In [24]:
model.feature_importances_

array([0.00044119, 0.        , 0.00094998, ..., 0.00112833, 0.00112845,
       0.        ])

In [25]:
#prediction = ohe.transform([["136-16"]])
#classifier.predict(prediction)

In [26]:
#classifier.save("dota_picks_trained.h5")